In [44]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn.functional import interpolate 
import matplotlib.pyplot as plt
from pathlib import Path
import time
# generation imports
from pippi.soundbuffer import SoundBuffer
from pippi import dsp,fx
import param_generation as pg
import _pickle as pickle
from IPython.display import Audio
from feature_extraction.mir_utils import *
###
from PIL import Image
###
import imp
import librosa
import librosa.display
import sys
import csv
from scipy.io.wavfile import write
import threading
from IPython.display import clear_output
sys.path.append("..")

from feature_extraction import resnet

import helpers
imp.reload(resnet)
imp.reload(helpers)
imp.reload(pg)
from feature_extraction import pytorch_models as tm

from helpers import *
# from common_vars import SR
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

drum_types = ['hat', 'kick', 'shake', 'snare']

# save_path="./generated_sample_pack/"

In [45]:
# good models
# cnet_dvn = tm.Transformer_DVN(attention_dropout=0.3,d_model=100,heads=20,encoding_layers=12,pool_dim=2,pct_start=0.05,max_lr=1e-4,max_momentum=0.95,epochs=50)
# cnet_dvn = cnet_dvn.load_from_checkpoint(
#     "models/transformer/dvn:epoch=10-val_auc=0.9987-val_accuracy=0.988-val_loss=0.039.ckpt")
# cnet_dvd = tm.Transformer_DVD(attention_dropout=0.3,d_model=100,heads=20,encoding_layers=12,pool_dim=2,pct_start=0.05,max_lr=1e-4,max_momentum=0.95,epochs=50)
# cnet_dvd = cnet_dvd.load_from_checkpoint(
#     "models/transformer/dvd:epoch=41-val_loss=0.247-val_auc=0.9712-val_accuracy=0.8948.ckpt")

In [48]:
imp.reload(tm)
cnet_dvn = tm.Transformer_DVN(attention_dropout=0.3,d_model=120,heads=30,encoding_layers=12,pool_dim=2,)
cnet_dvn = cnet_dvn.load_from_checkpoint(
    "models/transformer/DVNepoch=41-val_accuracy=0.991-val_loss=0.048.ckpt")

cnet_dvd = tm.Transformer_DVD(attention_dropout=0.3,d_model=100,heads=20,encoding_layers=12,pool_dim=1,pct_start=0.05,max_lr=1e-4,max_momentum=0.95,epochs=50)
cnet_dvd = cnet_dvd.load_from_checkpoint(
    "models/transformer/DVDepoch=91-val_accuracy=0.904-val_auc=0.9707-val_loss=0.383.ckpt")

In [118]:
cnet

NameError: name 'cnet' is not defined

In [49]:
SR = 44100
# make sound and evaluate
def make_and_eval(stack_size = 1):
    out,params = StackMaker(stack_size)
    a = memToAud(out)
    try:
        signal = torch.tensor(librosa.resample(a.reshape(-1),SR,SR//2))[0:SR//4]
    except Exception as e:
        signal = torch.tensor(np.zeros([1,1,SR//4])).float()
    with torch.no_grad():
        cnet_dvn.eval()
        ps_dvn = cnet_dvn(signal.reshape(1,-1))
        ps_dvd = cnet_dvd(signal.reshape(1,-1))
        
#         print(dict(zip(drum_types,ps_dvd[0].numpy())))
    return a,signal,ps_dvn,ps_dvd

synth_out,a,ps,ps_dvd = make_and_eval()
print(drum_types[np.argmax(ps_dvd)],ps)

a = a.reshape([-1])

print(len(a))
Audio(a,rate=SR//2,autoplay=True)

hat tensor([[-6.9353]])
11025


In [53]:
# loop until found

biggest_p = []
best_sounds = {}
for i in range(100):
    synth_out,a,ps,ps_dvd = make_and_eval(3)
    print(ps[0].item(),i,end=" \r")
#     print(drum_types[np.argmax(ps_dvd)])
    if ps[0] > 1 and a.sum():
        clear_output()
        print("\n",drum_types[np.argmax(ps_dvd)])
        break
    biggest_p.append(ps[0])
a = a.reshape([-1]).cpu().numpy()[0:SR//4]
anorm =  librosa.util.normalize(a)
Audio(anorm[0:SR//6].reshape([-1]),rate=SR//2,autoplay=True)


In [119]:
# loop until found

biggest_p = []
best_sounds = {}
for i in range(5000):
    synth_out,a,ps,ps_dvd = make_and_eval(np.random.choice([1,3,5]))
    print(ps[0].item(),i,end=" \r")
#     print(drum_types[np.argmax(ps_dvd)])
    if ps[0] > 0.5 and a.sum():
        write('loops/kits/%.4f:%s.wav'%(ps[0],drum_types[np.argmax(ps_dvd)]), 
                                      44100, synth_out[0:SR//2])
        print("\n type:",drum_types[np.argmax(ps_dvd)])
    biggest_p.append(ps[0])
# a = synth_out.reshape([-1])[0:SR//4]
a = a.reshape([-1]).cpu().numpy()[0:SR//4]
anorm =  librosa.util.normalize(a)
Audio(anorm[0:SR//6].reshape([-1]),rate=SR//2,autoplay=True)


0.7504149079322815 2542   
 type: hat
6.190188884735107 2608    
 type: snare
2.3530147075653076 2923  2 
 type: snare
6.190117359161377 2935 4 
 type: kick
4.690552711486816 3001    
 type: snare
1.1748387813568115 3022  
 type: hat
6.189702033996582 3078    
 type: hat
2.9681973457336426 3152  
 type: kick
1.134226679801941 3218    
 type: kick
6.1899614334106445 3275  
 type: hat
1.6085138320922852 3323  
 type: kick
0.6448178291320801 3508    
 type: hat
6.189327239990234 3517  
 type: shake
0.9573786854743958 3585  
 type: kick
1.3690606355667114 3682   
 type: snare
1.3820844888687134 3808  
 type: hat
2.3092586994171143 3823 
 type: kick
0.6020423769950867 3841   
 type: kick
0.743998110294342 3923   
 type: hat
6.190080642700195 3974    
 type: kick
0.5444779396057129 4161  
 type: hat
2.273015260696411 4371    
 type: hat
6.189273357391357 4404   
 type: snare
2.4490978717803955 4502  
 type: hat
6.190191745758057 4602    
 type: snare
1.1748100519180298 4707  
 type: hat
0.68

In [28]:
Audio(anorm[0:SR//4].reshape([-1]),rate=SR//2,autoplay=True)

In [ ]:
%%time
# getting the best sound out of n tries

best_sounds = {}
best_sounds = dict.fromkeys(drum_types)
for k in drum_types:
    best_sounds[k] = (0,[0])
def f():
    for i in range(100):
        print(i,end="\r")
        synth_out,a,ps,ps_dvd = make_and_eval(4)
        if ps[0]>-10:
            t,p = drum_types[np.argmax(ps_dvd)],ps_dvd[0][np.argmax(ps_dvd)]
            if best_sounds[t][0]<p:
                best_sounds[t] = (p,a)
    return True
x = f()

In [ ]:

# # playing a loop while drum kit is updated in seperate thread
drum_kit = {}
drum_kit = dict.fromkeys(drum_types)
counts = dict.fromkeys(drum_types)
for k in counts:
    counts[k] = 0
    
bpm = 120
num_beats = 8

def make_beat(drum_kit):
    loop = np.zeros(num_beats*30*SR//bpm ) # init silent loop
    loop = add_to_loop(loop,"kick",0,drum_kit)
    loop = add_to_loop(loop,"hat",1,drum_kit)
    loop = add_to_loop(loop,"snare",2,drum_kit)
    loop = add_to_loop(loop,"kick",3,drum_kit)
    loop = add_to_loop(loop,"hat",4,drum_kit)
    loop = add_to_loop(loop,"snare",5,drum_kit)
    loop = add_to_loop(loop,"kick",6,drum_kit)
    loop = add_to_loop(loop,"shake",7,drum_kit)
    return loop

# bpm = 300
# num_beats = 7
# def make_beat(drum_kit):
#     loop = np.zeros(num_beats*60*SR//bpm ) # init silent loop
#     loop = add_to_loop(loop,"kick",0,drum_kit)
#     loop = add_to_loop(loop,"hat",1,drum_kit)
#     loop = add_to_loop(loop,"kick",2,drum_kit)
#     loop = add_to_loop(loop,"snare",4,drum_kit)
#     loop = add_to_loop(loop,"shake",5,drum_kit)
#     return loop

def add_to_loop(loop,drum,beat,drum_kit):
        # given a loop, it puts a drum at beat n
        beat_position = int(SR*beat*30//bpm)
        try:
            drum_sound =  drum_kit[drum][1][0:SR//4]
        except:
            drum_sound = np.zeros(SR)
        start,stop = beat_position,min(beat_position+len(drum_sound),len(loop))
        loop[start:stop] += drum_sound[0:stop-start]
        return loop

def play_loop(loop,sr=SR):
        ipd.set_matplotlib_close(close=True)
        ipd.display(ipd.Audio(loop, rate=sr,autoplay=True))
        loop_length = len(loop)/sr
        time.sleep(loop_length)
        clear_output()


def kit_updater(drum_kit,counts,tries=50,):
    # updates a drum kit for n tries
    loops_made = 0
    for i in range(tries):
        synth_out,a,ps,ps_dvd =  make_and_eval(4)
        if ps[0] > 0 :
            t,p = drum_types[np.argmax(ps_dvd)],ps_dvd[0][np.argmax(ps_dvd)]
#             a = mix_down(a,t)
            drum_kit[t] = (p,synth_out)
            counts[t] += 1
            loop = make_beat(drum_kit)
#             write('loops/mantra/loop:%d-attempt:%d.wav'%(loops_made,i), 44100, loop)
            loops_made+=1
        if loops_made>100:
            break

def morph_kit():
    kit_updater_thread = threading.Thread(target=kit_updater,args=(drum_kit,counts,1000))
    kit_updater_thread.start()
    while kit_updater_thread.is_alive():
        print(drum_kit)
        loop = make_beat(drum_kit)
        ipd.set_matplotlib_close(close=True)
        ipd.display(ipd.Audio(loop, rate=SR,autoplay=True))
        loop_length = len(loop)/SR
        time.sleep(loop_length)
        clear_output()
# loop = make_beat()
# play_loop(loop)
morph_kit()

In [ ]:
def kit_updater(drum_kit,tries=10,):
    for i in range(tries):
        a,a_ds,ps,pss,pss_dvd= make_and_eval(1)
        if pss[0]>pss[1]:
            t,p = drum_types[np.argmax(pss_dvd)],pss_dvd[np.argmax(pss_dvd)]
            drum_kit[t] = (p,a)
        
def check_kit():
    kit_updater_thread = threading.Thread(target=kit_updater,args=(drum_kit,500))
    kit_updater_thread.start()
    while kit_updater_thread.is_alive():
        time.sleep(1)
check_kit()

In [12]:
bpm = 300
num_beats = 7
def make_beat(drum_kit):
    loop = np.zeros(num_beats*60*SR//bpm ) # init silent loop
    loop = add_to_loop(loop,"kick",0,drum_kit)
    loop = add_to_loop(loop,"hat",1,drum_kit)
    loop = add_to_loop(loop,"kick",2,drum_kit)
    loop = add_to_loop(loop,"snare",4,drum_kit)
    loop = add_to_loop(loop,"shake",5,drum_kit)
    return loop
loop = make_beat(best_sounds)
ipd.display(ipd.Audio(loop, rate=SR,autoplay=True))

TypeError: Concatenation operation is not implemented for NumPy arrays, use np.concatenate() instead. Please do not rely on this error; it may not be given on all Python implementations.

In [18]:
likes = pd.read_csv("csvs/drum_track_abram.txt")

In [26]:
likes.columns=["loop","bad","c1","c2","c3"]
likes[0:99]["bad"].sum(),likes[99:]["bad"].sum()

(54, 55)

In [ ]:
def mix_down(a,cat):
    a = librosa.util.normalize(a)
    if cat=="kick":
        a = helpers.butter_bandpass_filter(a,30,8000,48000,order=2)
        a = librosa.util.normalize(a)
    if cat == "shake":
        a = helpers.butter_bandpass_filter(a,10000,15000,48000,order=2)
        a = librosa.util.normalize(a)
        a = a * 0.6
    if cat == "hat":
        a = helpers.butter_bandpass_filter(a,8000,15000,48000,order=2)
        a = librosa.util.normalize(a)
        a = a * 0.9
    if cat == "snare":
        a = helpers.butter_bandpass_filter(a,300,12000,48000,order=2)
        a = librosa.util.normalize(a)
        a = a * 0.9
        a = a.reshape([-1])
    return a